### Concatination and manipulation of files


In [1]:
import matplotlib.pyplot as plt
import sys, os
from math import floor,ceil
import numpy as np
from netCDF4 import Dataset
import xarray as xr
import cmocean 
import pandas as pd
import datetime
from matplotlib import dates as mdates

from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [40]:
PATH = '/content/drive/MyDrive/testing'

test1 = '/content/drive/MyDrive/testing/AQUA_MODIS.20110201_20110228.L3m.MO.CHL.chlor_a.4km.nc'
ncfile = Dataset(test1,"r+", format="NETCDF4") # Remember to use r+ for append

OSError: ignored

In [41]:
ncfile

<class 'netCDF4._netCDF4.Dataset'>
root group (NETCDF4 data model, file format HDF5):
    product_name: AQUA_MODIS.20110201_20110228.L3m.MO.CHL.chlor_a.4km.nc
    instrument: MODIS
    title: MODISA Level-3 Standard Mapped Image
    project: Ocean Biology Processing Group (NASA/GSFC/OBPG)
    platform: Aqua
    source: satellite observations from MODIS-Aqua
    temporal_range: month
    processing_version: R2022.0
    date_created: 2022-07-25T19:37:18.000Z
    history: l3mapgen par=AQUA_MODIS.20110201_20110228.L3m.MO.CHL.chlor_a.4km.nc.param 
    l2_flag_names: ATMFAIL,LAND,HILT,HISATZEN,STRAYLIGHT,CLDICE,COCCOLITH,LOWLW,CHLWARN,CHLFAIL,NAVWARN,MAXAERITER,ATMWARN,HISOLZEN,NAVFAIL,FILTER,HIGLINT
    time_coverage_start: 2011-02-01T00:50:00.000Z
    time_coverage_end: 2011-03-01T02:59:59.000Z
    start_orbit_number: 46524
    end_orbit_number: 46933
    map_projection: Equidistant Cylindrical
    latitude_units: degrees_north
    longitude_units: degrees_east
    northernmost_latitude: 9

In [6]:
# Creating a dimension time
time_dim = ncfile.createDimension('time', None) # unlimited axis (can be appended to).
for dim in ncfile.dimensions.items():
  print(dim)

('lat', <class 'netCDF4._netCDF4.Dimension'>: name = 'lat', size = 4320)
('lon', <class 'netCDF4._netCDF4.Dimension'>: name = 'lon', size = 8640)
('rgb', <class 'netCDF4._netCDF4.Dimension'>: name = 'rgb', size = 3)
('eightbitcolor', <class 'netCDF4._netCDF4.Dimension'>: name = 'eightbitcolor', size = 256)
('time', <class 'netCDF4._netCDF4.Dimension'> (unlimited): name = 'time', size = 0)


In [16]:
ncfile['chlor_a']

<class 'netCDF4._netCDF4.Variable'>
float32 chlor_a(lat, lon)
    long_name: Chlorophyll Concentration, OCI Algorithm
    units: mg m^-3
    standard_name: mass_concentration_of_chlorophyll_in_sea_water
    _FillValue: -32767.0
    valid_min: 0.001
    valid_max: 100.0
    reference: Hu, C., Lee Z., and Franz, B.A. (2012). Chlorophyll-a algorithms for oligotrophic oceans: A novel approach based on three-band reflectance difference, J. Geophys. Res., 117, C01011, doi:10.1029/2011JC007395.
    display_scale: log
    display_min: 0.01
    display_max: 20.0
unlimited dimensions: 
current shape = (4320, 8640)
filling on

In [9]:
# Creating a variable
time = ncfile.createVariable('time', np.float64, ('time',))
time.units = 'hours since 2011-02-01'
time.long_name = 'time'

RuntimeError: ignored

In [44]:
import datetime as dt
from netCDF4 import date2num,num2date
dates = dt.datetime(2011,2,1,0)
time = date2num(dates, time.units)

ValueError: ignored

In [15]:
variable = ncfile['chlor_a']
for attrname in variable.ncattrs():
  print("{} -- {}".format(attrname, variable.getncattr(attrname)))

long_name -- Chlorophyll Concentration, OCI Algorithm
units -- mg m^-3
standard_name -- mass_concentration_of_chlorophyll_in_sea_water
_FillValue -- -32767.0
valid_min -- 0.0010000000474974513
valid_max -- 100.0
reference -- Hu, C., Lee Z., and Franz, B.A. (2012). Chlorophyll-a algorithms for oligotrophic oceans: A novel approach based on three-band reflectance difference, J. Geophys. Res., 117, C01011, doi:10.1029/2011JC007395.
display_scale -- log
display_min -- 0.009999999776482582
display_max -- 20.0


In [23]:
temp = ncfile.createVariable('temp2',np.float64,('time','lat','lon'),fill_value=-32767.0) # note: unlimited dimension is leftmost

# adding attributes
for attrname in variable.ncattrs():
  if(attrname != "_FillValue"):
    temp.setncattr(attrname, variable.getncattr(attrname))

print(temp)

<class 'netCDF4._netCDF4.Variable'>
float64 temp2(time, lat, lon)
    _FillValue: -32767.0
    long_name: Chlorophyll Concentration, OCI Algorithm
    units: mg m^-3
    standard_name: mass_concentration_of_chlorophyll_in_sea_water
    valid_min: 0.001
    valid_max: 100.0
    reference: Hu, C., Lee Z., and Franz, B.A. (2012). Chlorophyll-a algorithms for oligotrophic oceans: A novel approach based on three-band reflectance difference, J. Geophys. Res., 117, C01011, doi:10.1029/2011JC007395.
    display_scale: log
    display_min: 0.01
    display_max: 20.0
unlimited dimensions: time
current shape = (0, 4320, 8640)
filling on


In [29]:
chlor_a_values = ncfile['chlor_a'][:,:]

In [31]:
temp[0,:,:] = chlor_a_values

In [32]:
print(ncfile)
# close the Dataset.
ncfile.close(); print('Dataset is closed!')

<class 'netCDF4._netCDF4.Dataset'>
root group (NETCDF4 data model, file format HDF5):
    product_name: AQUA_MODIS.20110201_20110228.L3m.MO.CHL.chlor_a.4km.nc
    instrument: MODIS
    title: MODISA Level-3 Standard Mapped Image
    project: Ocean Biology Processing Group (NASA/GSFC/OBPG)
    platform: Aqua
    source: satellite observations from MODIS-Aqua
    temporal_range: month
    processing_version: R2022.0
    date_created: 2022-07-25T19:37:18.000Z
    history: l3mapgen par=AQUA_MODIS.20110201_20110228.L3m.MO.CHL.chlor_a.4km.nc.param 
    l2_flag_names: ATMFAIL,LAND,HILT,HISATZEN,STRAYLIGHT,CLDICE,COCCOLITH,LOWLW,CHLWARN,CHLFAIL,NAVWARN,MAXAERITER,ATMWARN,HISOLZEN,NAVFAIL,FILTER,HIGLINT
    time_coverage_start: 2011-02-01T00:50:00.000Z
    time_coverage_end: 2011-03-01T02:59:59.000Z
    start_orbit_number: 46524
    end_orbit_number: 46933
    map_projection: Equidistant Cylindrical
    latitude_units: degrees_north
    longitude_units: degrees_east
    northernmost_latitude: 9

In [36]:
ncfile.to_netcdf(path ="./new.nc")

AttributeError: ignored

In [37]:
dataset = xr.open_dataset("/content/drive/MyDrive/testing/AQUA_MODIS.20110201_20110228.L3m.MO.CHL.chlor_a.4km.nc")

In [38]:
dataset

<xarray.Dataset>
Dimensions:  (lat: 4320, lon: 8640, rgb: 3, eightbitcolor: 256, time: 1)
Coordinates:
  * lat      (lat) float32 89.98 89.94 89.9 89.85 ... -89.85 -89.9 -89.94 -89.98
  * lon      (lon) float32 -180.0 -179.9 -179.9 -179.9 ... 179.9 179.9 180.0
  * time     (time) float64 9.969e+36
Dimensions without coordinates: rgb, eightbitcolor
Data variables:
    chlor_a  (lat, lon) float32 ...
    palette  (rgb, eightbitcolor) uint8 ...
    CHLOR_A  (time, lat, lon) float64 ...
    temp     (time, lat, lon) float64 ...
    temp1    (time, lat, lon) float64 ...
    temp2    (time, lat, lon) float64 ...
Attributes: (12/62)
    product_name:                     AQUA_MODIS.20110201_20110228.L3m.MO.CHL...
    instrument:                       MODIS
    title:                            MODISA Level-3 Standard Mapped Image
    project:                          Ocean Biology Processing Group (NASA/GS...
    platform:                         Aqua
    source:                           satellite observations from MODIS-Aqua
    ...                               ...
    cdm_data_type:                    grid
    keywords:                         Earth Science > Oceans > Ocean Chemistr...
    keywords_vocabulary:              NASA Global Change Master Directory (GC...
    data_bins:                        19362815
    data_minimum:                     0.0025527535
    data_maximum:                     86.16987

In [42]:
time_variable = ncfile['time']
time_variable

<class 'netCDF4._netCDF4.Variable'>
float64 time(time)
    units: days from 2011-02-01
    long_name: time
unlimited dimensions: time
current shape = (1,)
filling on, default _FillValue of 9.969209968386869e+36 used